In [91]:
"""
imports
"""
# import mindspore
import pandas as pd
import numpy as np
import random
import os
from collections import defaultdict
from time import time
from tqdm import tqdm
import time
import argparse
import scipy.sparse as sp
import os
import mindspore as ms
from mindspore import nn
from mindspore.common.initializer import initializer
from mindspore.common.parameter import ParameterTuple
from mindspore.ops import composite as C
from mindspore.ops import functional as F
from mindspore.ops import operations as P
from mindspore import Tensor
from mindspore.nn.layer.activation import get_activation
import mindspore.context as context

In [92]:
"""
load data
"""
class Data(object):
    def __init__(self,npzpath="./data/viedo10/video10.npz"):

        self.user_item = defaultdict(set)
        self.item_user = defaultdict(set)

        self.user_vali_item = dict()
        self.user_test_item = dict()

        _data = np.load(npzpath, allow_pickle=True)
        self.train_data = _data['train_data']
        self.test_data = _data['test_data'].tolist()
        vali_data = _data['vali_data'].tolist()

        # todo consider using os.path.join
        p = npzpath.split('/')
        self.path = p[0] + '/' + p[1] + '/' + p[2]

        self.n_users, self.n_items = self.train_data.max(axis=0) + 1
        self.R = sp.dok_matrix((self.n_users, self.n_items), dtype=np.float32)

        for u, i in self.train_data:
            self.user_item[u].add(i)
            self.item_user[i].add(u)

            self.R[u, i] = 1.

        self.train_number = np.shape(self.train_data)[0]
        print(self.n_users, self.n_items,self.train_number, self.train_number/(self.n_users*self.n_items))

        for u in self.test_data.keys():
            self.user_test_item[u]=[self.test_data[u][0]]
            self.user_test_item[u].extend(self.test_data[u][1])

        for u in vali_data.keys():
            self.user_vali_item[u] = [vali_data[u][0]]
            self.user_vali_item[u].extend(vali_data[u][1])

        # self.nodesum = self.get_nodesum(depth)
    def gen_batch_train_data(self, neg_number, batch_size):
        np.random.shuffle(self.train_data)
        batch = np.zeros((batch_size, 3), dtype=np.uint32)
        idx = 0
        for u,i in self.train_data:
            for neg_num in range(neg_number):
                neg_item = random.randint(0, self.n_items - 1)
                while (neg_item in self.user_item[u]):
                    neg_item = random.randint(0, self.n_items  - 1)
                batch[idx, :] = [u,i, neg_item]
                idx += 1

                if (idx == batch_size):
                    yield batch
                    idx = 0

        if (idx > 0):
            yield batch[:idx]
    def gen_batch_test_data(self, test_neg_number, data='test'):
        size = test_neg_number + 1
        batch = np.zeros((size, 2), dtype=np.uint32)

        idx = 0
        if(data=='test'):
            for user, items in self.user_test_item.items():
                for item in items:
                    batch[idx, :] = [user, item]
                    idx += 1

                yield items[0], batch
                idx = 0

        elif(data=='vali'):
            for user, items in self.user_vali_item.items():
                for item in items:
                    batch[idx, :] = [user, item]
                    idx += 1

                yield items[0], batch
                idx = 0
        else:
            print("data type error.")
            exit(-1)
    def get_adj_mat(self):
        try:
            t1 = time.time()
            mean_adj_mat = sp.load_npz(self.path + '/s_mean_adj_mat.npz')
            print('already load adj matrix', mean_adj_mat.shape, time.time() - t1)

        except Exception:
            mean_adj_mat = self.create_adj_mat()
            sp.save_npz(self.path + '/s_mean_adj_mat.npz', mean_adj_mat)

        return  mean_adj_mat
    def get_adj_mat_nonorm(self):
        # try:
        #     t1 = time()
        #     adj_mat = sp.load_npz(self.path + '/adj_mat.npz')
        #     print('already load adj matrix', adj_mat.shape, time() - t1)

        # except Exception:
        adj_mat = sp.dok_matrix((self.n_users + self.n_items, self.n_users + self.n_items), dtype=np.float32)
        adj_mat = adj_mat.tolil()
        R = self.R.tolil()
        adj_mat[:self.n_users, self.n_users:] = R
        adj_mat[self.n_users:, :self.n_users] = R.T

        rowsum = np.array(adj_mat.sum(1)).flatten()
        d_mat_inv = sp.diags(rowsum)

        adj_mat = adj_mat+d_mat_inv

        adj_mat = adj_mat.tocsr()
        sp.save_npz(self.path + '/adj_mat.npz', adj_mat)

        return adj_mat
    def get_nodesum(self,depth):
        adj_mat = self.get_adj_mat_nonorm()
        edge_mat = adj_mat.dot(adj_mat)
        for i in range(depth-1):
            if(i!=0):
                edge_mat = edge_mat.dot(adj_mat)
            else:
                pass
        nodesum = edge_mat.sum(1).flatten()
        return nodesum
    def create_adj_mat(self):
        t1 = time.time()
        adj_mat = sp.dok_matrix((self.n_users+self.n_items, self.n_users+self.n_items), dtype=np.float32)
        adj_mat = adj_mat.tolil()
        R = self.R.tolil()

        adj_mat[:self.n_users, self.n_users:] = R
        adj_mat[self.n_users:, :self.n_users] = R.T
        adj_mat = adj_mat.todok()
        print('already create adjacency matrix', adj_mat.shape, time.time() - t1)

        t2 = time.time()

        def normalized_adj_single(adj):
            rowsum = np.array(adj.sum(1))

            d_inv = np.power(rowsum, -1).flatten()
            d_inv[np.isinf(d_inv)] = 0.
            d_mat_inv = sp.diags(d_inv)

            norm_adj = d_mat_inv.dot(adj)
            # norm_adj = adj.dot(d_mat_inv)
            print('generate single-normalized adjacency matrix.')
            return norm_adj.tocoo()

        mean_adj_mat = normalized_adj_single(adj_mat)

        print('already normalize adjacency matrix', time.time() - t2)
        return mean_adj_mat.tocsr()

In [93]:
def leave_one_out(purchased_item, recommend_list, top_k_recommand_number):
    top_recommend_list=recommend_list[:top_k_recommand_number]
    if (purchased_item in top_recommend_list):
        return 1, np.log2(2.0) / np.log2(top_recommend_list.index(purchased_item) + 2.0)
    else:
        return 0, 0
def NDCG_k(recommend_list, purchased_list):
    Z_u = 0
    temp=0
    for j in range(min(len(recommend_list), len(purchased_list))):
        Z_u = Z_u + 1 / np.log2(j + 2)
    for j in range(len(recommend_list)):
        if recommend_list[j] in purchased_list:
            temp = temp + 1 / np.log2(j + 2)
    if Z_u == 0:
        temp = 0
    else:
        temp = temp / Z_u
    return temp
def top_k(recommend_list, purchased_list):
    temp = []
    for j in recommend_list:
        if j in purchased_list:
            temp.append(j)
    if len(temp):
        HR = 1
    else:
        HR = 0
    co_length=len(temp)
    re_length=len(recommend_list)
    pu_length=len(purchased_list)

    if re_length == 0:
        p = 0.0
    else:
        p = co_length / float(re_length)

    if pu_length == 0:
        r = 0.0
    else:
        r = co_length / float(pu_length)

    if r != 0 or p != 0:
        f=2.0 * p * r / (p + r)
    else:
        f=0.0
    return p, r, f, HR

In [94]:
class params:
    test_user_number = 0
    neg_number = 1
    test_neg_number = 100
    learning_rate = 0.0001
    batch_size = 1024
    pretrain = 10
    learner = "adam"
    n_fold = 1
    mess_dropout = 0.0
    node_dropout = 0.1
    depth = 10
    alpha = 0.5
    loss = 0
    l2_regeularization = 0.0001
    number_users = 0
    number_items = 0
    global_dimention = 50
    verbose = 1
    note = 'edge-simi'
    edge = 'add'
    save = 1
    outward = 0.5
    epochs = 500

In [95]:
"""
define the model
"""
class LECF(nn.Cell):
    def __init__(self,data):
        super(LECF,self).__init__(auto_prefix=True)
        # create trainable parameters
        self.data = data
        self.user_embedding_weight = ms.Parameter(default_input=initializer('XavierUniform',[params.number_users,params.global_dimention] ,ms.float32),name="item_embedding_matrix",requires_grad=True,layerwise_parallel=False)
        self.item_embedding_weight = ms.Parameter(default_input=initializer('XavierUniform',[params.number_items,params.global_dimention] ,ms.float32),name="item_embedding_matrix",requires_grad=True,layerwise_parallel=False)
        self.edge_weight = ms.Parameter(default_input=initializer('XavierUniform',[2 * params.global_dimention,params.global_dimention] ,ms.float32),name="edge_weight",requires_grad=True,layerwise_parallel=False)
        self.dl = 1
        if (params.edge == 'concat'): self.dl = 2
        self.test_user_g_embeddings = ms.Parameter(default_input=initializer('ones',shape=[params.number_users,params.global_dimention * self.dl] , dtype=ms.float32),name='test_user_g_embeddings',requires_grad=True,layerwise_parallel=False)
        self.test_item_g_embeddings = ms.Parameter(default_input=initializer('ones',shape=[params.number_items,params.global_dimention * self.dl] , dtype=ms.float32),name='test_item_g_embeddings',requires_grad=True,layerwise_parallel=False)

        # initializing part of the trainable parameters
        self.A_fold_hat_c = self._get_fold_hat(params.outward)
        self.A_fold_hat_e = self._get_fold_hat(-1)
        self.concat0 = P.Concat(axis=0)
        self.concat1 = ms.ops.Concat(axis=0)
        self.concat2 = P.Concat(axis=0)
        self.ego_embeddings = self.concat0((self.user_embedding_weight,self.item_embedding_weight))
        print(self.user_embedding_weight.shape,"::::", self.item_embedding_weight.shape,"::::",self.ego_embeddings.shape)
        self.matmul = P.MatMul(transpose_a=False,transpose_b=False)
        for K in range(params.depth):
            if (K == 0):
                self.A_fold_hat = self.A_fold_hat_e
            else : 
                self.A_fold_hat = self.A_fold_hat_c
            
            temp_embed = []
            for G in range(params.n_fold):
                # print("G is: ",G,", A_fold_hat is",self.A_fold_hat[G].shape)
                temp_embed.append(self.matmul(Tensor(self.A_fold_hat[G],ms.float32),self.ego_embeddings))
            
            if (K == 0):
                if (params.edge == "add"):
                    # todo: figure what happened here.
                    # self.ego_embeddings += self.concat1(temp_embed)
                    self.ego_embeddings += temp_embed[0]
                else:
                    pass
                # todo: other edges.
            else: 
                # self.ego_embeddings = self.concat2((temp_embed,0))
                self.ego_embeddings = temp_embed[0]
            self.dropout = nn.Dropout(keep_prob=1-params.mess_dropout)

            if (params.mess_dropout != 0):
                self.ego_embeddings = self.dropout(self.ego_embeddings)
        # the network logic
        self.l2_normalize = P.L2Normalize(axis=0,epsilon=1e-4)
        self.split = P.Split(axis=0,output_num=2)
        self.embedding_lookup_FUE = nn.EmbeddingLookup(vocab_size=self.user_embedding_weight.shape[0], \
                        embedding_size=self.user_embedding_weight.shape[1] , param_init=self.user_embedding_weight,target="CPU",slice_mode="batch_slice")
        self.embedding_lookup_FIE = nn.EmbeddingLookup(vocab_size=self.item_embedding_weight.shape[0], \
                        embedding_size=self.item_embedding_weight.shape[1] , param_init=self.item_embedding_weight,target="CPU",slice_mode="batch_slice")
        self.embedding_lookup_FNIE = nn.EmbeddingLookup(vocab_size=self.item_embedding_weight.shape[0], \
                        embedding_size=self.item_embedding_weight.shape[1] , param_init=self.item_embedding_weight,target="CPU",slice_mode="batch_slice")

        self.reduce_sum_y = P.ReduceSum(keep_dims=True)
        self.reduce_sum_neg_y = P.ReduceSum(keep_dims=True)
        self.matmul_y = P.Mul()
        self.matmul_neg_y = P.Mul()


    def construct(self, ID):
        """
        the forward pass route.
        :param ID: input batch data.
        :return: the solution.
        """
        user_id = ID[:,0]
        item_id = ID[:,1]
        neg_item_id = ID[:,2]
        # print("user_id is: ",ID[:,0].shape," item_id is: ",ID[:,1].shape," neg_item_id is: ",ID[:,2].shape)

        # norm_embeddings = self.l2_normalize(ego_embeddings)
        # users_g_embeddings, items_g_embeddings = self.split(self.ego_embeddings, [int(self.data.n_users), int(self.data.n_items)])

        users_g_embeddings = self.ego_embeddings[:int(self.data.n_users),:]
        items_g_embeddings = self.ego_embeddings[int(self.data.n_users):,:]
        # print(self.data.n_items, self.data.n_users)
        # print("the shape of g_e is:", self.ego_embeddings.shape," u_g_e is: ",users_g_embeddings.shape," i_g_e is: ",items_g_embeddings.shape)
        first_user_embedding = self.embedding_lookup_FUE(ms.ops.Cast()(user_id,ms.int32))
        # print('the shape of first_usr_embedding is: ',first_user_embedding.shape," user_id's shape is: ",user_id.shape)
        first_item_embedding = self.embedding_lookup_FIE(ms.ops.Cast()(item_id,ms.int32))
        first_neg_item_embedding = self.embedding_lookup_FNIE(ms.ops.Cast()(neg_item_id,ms.int32))

        self.embedding_lookup_LUE = nn.EmbeddingLookup(vocab_size=users_g_embeddings.shape[0], \
                        embedding_size=users_g_embeddings.shape[1] , param_init=users_g_embeddings,target="CPU",slice_mode="batch_slice")
        self.embedding_lookup_LIE = nn.EmbeddingLookup(vocab_size=items_g_embeddings.shape[0], \
                        embedding_size=items_g_embeddings.shape[1] , param_init=items_g_embeddings,target="CPU",slice_mode="batch_slice")
        self.embedding_lookup_LNIE = nn.EmbeddingLookup(vocab_size=items_g_embeddings.shape[0], \
                        embedding_size=items_g_embeddings.shape[1] , param_init=items_g_embeddings,target="CPU",slice_mode="batch_slice")

        last_user_embedding = self.embedding_lookup_LUE(ms.ops.Cast()(user_id,ms.int32))
        last_item_embedding = self.embedding_lookup_LIE(ms.ops.Cast()(item_id,ms.int32))
        last_neg_item_embedding = self.embedding_lookup_LNIE(ms.ops.Cast()(neg_item_id,ms.int32))

        query_pair = first_user_embedding + first_item_embedding
        neg_query_pair = first_user_embedding + first_neg_item_embedding

        # if(params.y == 'edge'):
        y = self.reduce_sum_y(self.matmul_y(query_pair,last_user_embedding+last_item_embedding),(1,))

        neg_y = self.reduce_sum_neg_y(self.matmul_neg_y(neg_query_pair,last_user_embedding+last_neg_item_embedding),(1,))

        return y, neg_y, first_user_embedding, first_item_embedding,first_neg_item_embedding
        # return 0 # for test
    def _get_fold_hat(self, outward):
        mean_adj_mat = self.data.get_adj_mat()
        A_fold_hat = []
        if(outward==-1):
            mat = 0.5*mean_adj_mat
        else:
            mat= outward*mean_adj_mat + (1-outward)*sp.eye(mean_adj_mat.shape[0])
        fold_len = (self.data.n_users + self.data.n_items) // params.n_fold # // operation is rounding down div.
        for i_fold in range(params.n_fold):
            start = i_fold * fold_len
            if (i_fold == params.n_fold - 1):
                end = self.data.n_users + self.data.n_items
            else:
                end = (i_fold + 1) * fold_len
            temp = mat[start:end].todense().astype(np.float32)

            """
            # use the coordinate matrix, which is not supported on CPU.
            coo = mat[start:end].tocoo().astype(np.float32)
            indices = np.mat([coo.row, coo.col]).transpose()
            temp = ms.SparseTensor(indices=indices, values=coo.data, dense_shape=coo.shape)
            print(temp.values,temp.indices,temp.dense_shape)
            """

            """
            # ALL the CRUCIAL ops are not implemented in mindspore cpu.
            # todo: finish after developed.
            if (params.node_dropout != 0):
                random_tensor = 1 - params.node_dropout
                # original tf ops: random_tensor += tf.random_uniform([mat[start:end].count_nonzero()])
                # ms-Ascend ops(not supported on windows): random_tensor += ms.ops.uniform(shape=(mat[start:end].count_nonzero(),)\
                #       ,minval=Tensor(0.0,dtype=ms.float32),maxval=Tensor(1.0,dtype=ms.float32),seed=365,dtype=ms.float32)

                # original tf ops: dropout_mask = tf.cast(tf.floor(random_tensor), dtype=tf.bool)
                # ms-Ascend ops(not supported on windows): dropout_mask = ms.ops.cast(ms.ops.Floor(random_tensor), dtype=ms.bool_)

                # temp = tf.sparse_retain(temp, dropout_mask) * tf.div(1., 1 - self.node_dropout)
                # not implemented on mindspore.
            """

            A_fold_hat.append(temp)
        return A_fold_hat

In [96]:
# the logic of reading the dataset.

dataset_dir = "./data/video10/video10.npz"
data = Data(npzpath=dataset_dir)
params.test_user_number = len(list(data.user_test_item.keys()))
print("the test_user_number is ",params.test_user_number)
params.train_number = data.train_number * params.neg_number
print("the train_number is ", params.train_number)
params.number_users = int(data.n_users)
params.number_items = int(data.n_items)
print(type(params.number_items),"should")
print("the user number is: ",params.number_users,"\nthe item number is: ",params.number_items)


1372 7957 20437 0.001872033755781348
the test_user_number is  1372
the train_number is  20437
<class 'int'> should
the user number is:  1372 
the item number is:  7957


In [97]:
# part of testing "initializer" of Parameter
import mindspore as ms
from mindspore import Parameter
from mindspore.common.initializer import initializer
from mindspore.common.initializer import XavierUniform

context.set_context(mode=context.PYNATIVE_MODE,device_target="CPU")
init_a_embedding_value = ms.Parameter(default_input=initializer(init="XavierUniform", shape=[100, 100], dtype=ms.int32),name="embedding_matrix",requires_grad=True, layerwise_parallel=False)
print("the matrix is in size: ",init_a_embedding_value.shape," and value:",init_a_embedding_value)
# the shape parameter should be python int, numpy.int32 will lead to error.
# tramsform the data could simply use int(x).

the matrix is in size:  (100, 100)  and value: Parameter (name=embedding_matrix)


In [98]:
context.set_context(mode=context.PYNATIVE_MODE,device_target="CPU",save_graphs=False)
net = LECF(data=data)
print("the embeddings include: ",net.trainable_params())
progress = enumerate(data.gen_batch_train_data(params.neg_number, params.batch_size))
for k, e in progress:
    # LECF.set_train(mode=True)
    output = net(Tensor(e,ms.float32))
    print("the output shape of the net is: ", output[1].shape)

already load adj matrix (9329, 9329) 0.005982160568237305
already load adj matrix (9329, 9329) 0.003988981246948242
(1372, 50) :::: (7957, 50) :::: (9329, 50)
the embeddings include:  [Parameter (name=item_embedding_matrix), Parameter (name=item_embedding_matrix), Parameter (name=edge_weight), Parameter (name=embedding_lookup_FUE.embedding_table), Parameter (name=embedding_lookup_FIE.embedding_table), Parameter (name=embedding_lookup_FNIE.embedding_table), Parameter (name=embedding_lookup_LUE.embedding_table), Parameter (name=embedding_lookup_LIE.embedding_table), Parameter (name=embedding_lookup_LNIE.embedding_table)]
the output shape of the net is:  (1024, 1)
the output shape of the net is:  (1024, 1)
the output shape of the net is:  (1024, 1)
the output shape of the net is:  (1024, 1)
the output shape of the net is:  (1024, 1)
the output shape of the net is:  (1024, 1)
the output shape of the net is:  (1024, 1)
the output shape of the net is:  (1024, 1)
the output shape of the net i

In [99]:
class with_loss_cell(nn.Cell):
    def __init__(self, network):
        super(with_loss_cell,self).__init__(auto_prefix=True)
        self.network = network
        # initialize LOSS
        self.reduce_sum = P.ReduceSum()
        self.sigmoid = nn.Sigmoid()
        self.loss_fn_first_item = nn.Norm(axis=0,keep_dims=False)# use nn.Norm as a loss function.
        self.loss_fn_first_user = nn.Norm(axis=0,keep_dims=False)
        self.loss_fn_first_neg_item_embedding = nn.Norm(axis=0,keep_dims=False)
        self.log = P.Log()
        self.div = P.RealDiv()
        self.add = P.TensorAdd()

    def construct(self, x):
        y, neg_y, first_user_embedding, first_item_embedding,first_neg_item_embedding = self.network(x)
        mf_loss = self.reduce_sum(self.log(self.sigmoid(y - neg_y) + 1e-6))
        first_user_loss = self.loss_fn_first_user(first_user_embedding)
        first_item_loss = self.loss_fn_first_item(first_item_embedding)
        first_neg_item_embedding_loss = self.loss_fn_first_neg_item_embedding(first_neg_item_embedding)
        reg_loss = params.l2_regeularization * (first_neg_item_embedding_loss + first_item_loss +
            first_user_loss) / params.batch_size
        return mf_loss + reg_loss

In [100]:
class train_one_step_cell(nn.Cell):
    def __init__(self, network, optimizer, sens=1.0):
        super(train_one_step_cell, self).__init__(auto_prefix=True)
        self.network = network
        self.network.set_grad()
        self.network.add_flags(defer_inline=True)
        self.weights = ParameterTuple(network.trainable_params())
        self.optimizer = optimizer
        self.grad = C.GradOperation(get_by_list=True, sens_param=True)
        self.sens = sens

    def construct(self,x):
        weights = self.weights
        loss = self.network(x)
        sens = P.Fill()(P.DType()(loss), P.Shape()(loss), self.sens)
        grads = self.grad(self.network, weights)(x,sens)
        return F.depend(loss, self.optimizer(grads))


In [101]:
context.set_context(mode=context.PYNATIVE_MODE,device_target="CPU",save_graphs=False)
net = LECF(data=data)
loss = nn.Norm(axis=0,keep_dims=False)
net_with_loss = with_loss_cell(network=net)
# ms.nn.TrainOneStepCell
opt = nn.Adam(net.trainable_params())
net_train_one_step = train_one_step_cell(network=net_with_loss, optimizer=opt)
net_train_one_step.requires_grad = True
print("the original net's embeddings include: ",net.trainable_params())
print("all embeddings include: ",net_with_loss.trainable_params())
progress = enumerate(data.gen_batch_train_data(params.neg_number, params.batch_size))
net_train_one_step.set_train(mode=True)
for k, e in progress:
    # loss = net_train_one_step(Tensor(e,ms.float32))
    loss = net_with_loss(Tensor(e,ms.float32))
    print("the loss of the net is: ",loss," in step: ",k)
    exit(-1)

already load adj matrix (9329, 9329) 0.0029959678649902344
already load adj matrix (9329, 9329) 0.003988742828369141
(1372, 50) :::: (7957, 50) :::: (9329, 50)
the original net's embeddings include:  [Parameter (name=network.user_embedding_weight), Parameter (name=network.item_embedding_weight), Parameter (name=network.edge_weight), Parameter (name=network.embedding_lookup_FUE.embedding_table), Parameter (name=network.embedding_lookup_FIE.embedding_table), Parameter (name=network.embedding_lookup_FNIE.embedding_table), Parameter (name=network.embedding_lookup_LUE.embedding_table), Parameter (name=network.embedding_lookup_LIE.embedding_table), Parameter (name=network.embedding_lookup_LNIE.embedding_table)]
all embeddings include:  [Parameter (name=network.user_embedding_weight), Parameter (name=network.item_embedding_weight), Parameter (name=network.edge_weight), Parameter (name=network.embedding_lookup_FUE.embedding_table), Parameter (name=network.embedding_lookup_FIE.embedding_table),

[WARNING] ME(11224:5820,MainProcess):2021-02-26-00:36:59.672.213 [mindspore\ops\operations\math_ops.py:171] WARN_DEPRECATED: The usage of TensorAdd is deprecated. Please use Add.


RuntimeError: mindspore\ccsrc\runtime\device\cpu\kernel_select_cpu.cc:299 SetKernelInfo] Operator[MakeRowTensor] is not support. Trace: 
In file C:\Users\15561\.conda\envs\MS\lib\site-packages\mindspore\ops\_grad\grad_array_ops.py(271)/        return RowTensor(new_indices, actual_dout, x_shp), zeros_like(indices), zeros_like(offset)/


# 